In [2]:
L = 8
t = 1
function Hamiltonian(L,t)
    H = zeros(2*L, 2*L)
    H[1,L] = -t
    H[L,1] = -t
    H[2L,L+1] = -t
    H[L+1,2L] = -t
    for i=1:2*L
      for j=1:2*L
        if i == j + 1 || i == j - 1
          H[i,j] = -t
        end 
      end
    end
    H[L,L+1]=0
    H[L+1,L]=0
    return H
end

H = Hamiltonian(L,t)

16×16 Matrix{Float64}:
  0.0  -1.0   0.0   0.0   0.0   0.0  …   0.0   0.0   0.0   0.0   0.0   0.0
 -1.0   0.0  -1.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0  -1.0   0.0  -1.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0  -1.0   0.0  -1.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0  -1.0   0.0  -1.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0  -1.0   0.0  …   0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0  -1.0      0.0   0.0   0.0   0.0   0.0   0.0
 -1.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0  -1.0
  0.0   0.0   0.0   0.0   0.0   0.0     -1.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0  …   0.0  -1.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0     -1.0   0.0  -1.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0  -1.0   0.0  -1.0   0.0   0.0
  

In [3]:
using LinearAlgebra
U = eigvecs(H)  #unitary matrix
Udag = inv(U)   #U^-1
E = eigvals(H)  #eigenvalues of H

-2.0000000000000004

In [8]:
function U1(A, Ne) #take the first Ne columns
    U_1 = A[:,1:4]
    return U_1
end

U_1 = U1(U,4)

16×4 Matrix{Float64}:
 -0.353553   0.0        0.353553      0.0
 -0.353553   0.0        0.5           0.0
 -0.353553   0.0        0.353553      0.0
 -0.353553   0.0       -5.55112e-17   0.0
 -0.353553   0.0       -0.353553      0.0
 -0.353553   0.0       -0.5           0.0
 -0.353553   0.0       -0.353553      0.0
 -0.353553   0.0        0.0           0.0
  0.0       -0.353553   0.0           0.353553
  0.0       -0.353553   0.0           0.5
  0.0       -0.353553   0.0           0.353553
  0.0       -0.353553   0.0          -5.55112e-17
  0.0       -0.353553   0.0          -0.353553
  0.0       -0.353553   0.0          -0.5
  0.0       -0.353553   0.0          -0.353553
  0.0       -0.353553   0.0           0.0

In [14]:
state = [1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0]
function U_tilde(A,state)
    U_tild = []
    occupied_states = Vector{Int64}
    for i = 1 : 2*L
        if state[i] == 1
            append!( occupied_states , i )
        end
    end
    U_tild = A[occupied_states,:]
    return U_tild
end
U_tilde(U_1,state)

        

LoadError: ArgumentError: invalid index: 1.0 of type Float64

In [37]:
UT = U[[1,2,9,10],1:4]

4×4 Array{Float64,2}:
 -0.353553   0.0       0.353553  0.0
 -0.353553   0.0       0.5       0.0
  0.0       -0.353553  0.0       0.353553
  0.0       -0.353553  0.0       0.5

4×4 Array{Float64,2}:
  1.0          4.44089e-16   0.0          0.0
 -4.44089e-16  1.0           0.0          0.0
  0.0          0.0           1.0          4.44089e-16
  0.0          0.0          -4.44089e-16  1.0

In [66]:
function confi(L)
  N = L/2
  Ne = N/2 #number of electrons with a quarter of filling
    
state = []

for i in range(1,L):
  n = randint(0,L-i)
  state.append(electrons[n])
  electrons = np.delete(electrons, n)
print(state)
Hs = csr_matrix(H)
print(Hs)
states = rand(length(electronsup) + length(electronsdown) + length(ceros),N)
    end 

confi (generic function with 1 method)

In [67]:
zeros(Int64(4/2))

2-element Array{Float64,1}:
 0.0
 0.0

In [69]:
confi(L)

LoadError: [91mMethodError: no method matching rand(::Int32, ::Float64)[39m
[91m[0mClosest candidates are:[39m
[91m[0m  rand([91m::Random.AbstractRNG[39m, ::Any) at C:\buildbot\worker\package_win32\build\usr\share\julia\stdlib\v1.5\Random\src\Random.jl:253[39m
[91m[0m  rand([91m::Integer...[39m) at C:\buildbot\worker\package_win32\build\usr\share\julia\stdlib\v1.5\Random\src\Random.jl:277[39m
[91m[0m  rand([91m::Random.AbstractRNG[39m, ::Any, [91m::Tuple{Vararg{Int32,N}} where N[39m) at C:\buildbot\worker\package_win32\build\usr\share\julia\stdlib\v1.5\Random\src\Random.jl:279[39m
[91m[0m  ...[39m